In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from glob import glob

DATA_PATH = 'simulations.csv'
COLORS = {
    'random': '#B473FB',
    'circle': '#636EFA',
    'grid': '#EF553B',
    'normal': '#00CC96'
}

In [2]:
raw_simulations = pd.read_csv(DATA_PATH)
simulations = raw_simulations.groupby(['Topology', 'Perturbation speed']).agg('mean').reset_index()

simulations.head()

,Topology,Perturbation speed,Observers count,Grid size,Send probability,Drop probability,Perturbation radius,Created,Forwarded,Received,ARQ replies,Avg per log per relay received,Avg per log per relay ARQ replies,Average delays,Minimum delays,Maximum delays
0,circle,1,125.0,30.0,0.01,0.0,5.0,37.333333,4333.333333,166666.666667,3800.000000,34.850000,0.660000,75.660000,74.766667,95.070000
1,circle,2,125.0,30.0,0.01,0.0,5.0,31.333333,3833.333333,74333.333333,650.000000,18.980000,0.123333,30.493333,30.466667,30.516667
2,circle,3,125.0,30.0,0.01,0.0,5.0,32.000000,4066.666667,116666.666667,1533.333333,29.186667,0.313333,25.000000,25.000000,25.013333
3,circle,4,125.0,30.0,0.01,0.0,5.0,32.666667,4066.666667,75000.000000,400.000000,18.406667,0.060000,15.500000,15.500000,15.500000
4,circle,5,125.0,30.0,0.01,0.0,5.0,28.000000,3433.333333,78000.000000,230.000000,22.170000,0.046667,15.000000,15.000000,15.000000


In [3]:
def get_scatter_chart(topology, fields, title=''):
    tmp_df = raw_simulations.loc[raw_simulations['Topology'] == topology]
    tmp_df = tmp_df.groupby(['Perturbation speed']).agg('mean').sort_values(['Perturbation speed'], ascending=False).reset_index()
    fig = go.Figure()

    for field in fields:
        fig.add_trace(go.Scatter(x=tmp_df['Perturbation speed'], y=tmp_df[field], name=field, line_shape='spline', mode='lines'))

    fig.update_layout(
        title_text=title,
        xaxis=dict(
            title_text='Speed'
        ),
        yaxis=dict(
            title_text='Delay'
        )
    )

    # fig.write_image('{}-{}.png'.format(topology, 'delays'))

    return fig

def get_bar_chart(topology, field, title='', y_title='', y_range=[]):
    y_min = []
    y_max = []
    x = []

    for speed in range(10, 0, -1):
        tmp_df = raw_simulations.loc[raw_simulations['Topology'] == topology]
        tmp_df = tmp_df.loc[tmp_df['Perturbation speed'] == speed]

        max_received = tmp_df[field].max()
        min_received = tmp_df[field].min()

        y_min.append(min_received)
        y_max.append(max_received - min_received)
        x.append('{}'.format(speed))

    fig = go.Figure(data=[
        go.Bar(y=y_min, width=0, marker_color='rgb(255, 255, 255)', name='line'),
        go.Bar(y=y_max, marker_color=COLORS[topology], name=topology)
    ])

    fig.update_layout(
        showlegend=False,
        title_text=title,
        barmode='stack',
        xaxis=dict(
            tickmode='array',
            tickvals=[i for i in range(0, len(x))],
            ticktext=x,
            title_text='Speed'
        ),
        yaxis=dict(
            title_text=y_title,
            range=y_range
        )
    )

    # fig.write_image('{}-{}.png'.format(topology, field))

    return fig


def get_arq_replies_per_packet_bar_chart(topology, title=''):
    return get_bar_chart(topology, 'Avg per log per relay ARQ replies', title=title, y_title='Replies', y_range=[0, 4])


def get_propagations_per_packet_bar_chart(topology, title=''):
    return get_bar_chart(topology, 'Avg per log per relay received', title, y_title='Propagations', y_range=[0, 320])

### ARQ replies per packet

In [4]:
fig = px.line(simulations, x='Perturbation speed', y='Avg per log per relay ARQ replies', color='Topology', title='ARQ reply propagations per packet', line_shape='spline')
fig.update_yaxes(title_text='Propagations')
fig.update_xaxes(title_text='Speed')
fig.show()

In [5]:
fig = get_arq_replies_per_packet_bar_chart('random', title='ARQ replies per packet - random topology')
fig.show()

In [6]:
fig = get_arq_replies_per_packet_bar_chart('normal', title='ARQ replies per packet - normal topology')
fig.show()

In [7]:
fig = get_arq_replies_per_packet_bar_chart('grid', title='ARQ replies per packet - grid topology')
fig.show()

In [8]:
fig = get_arq_replies_per_packet_bar_chart('circle', title='ARQ replies per packet - circle topology')
fig.show()

### Propagations per packet

In [9]:
fig = px.line(simulations, x='Perturbation speed', y='Avg per log per relay received', color='Topology', title='Propagations per packet', line_shape='spline')
fig.update_yaxes(title_text='Propagations')
fig.update_xaxes(title_text='Speed')
fig.show()

In [10]:
fig = get_propagations_per_packet_bar_chart('random', title='Propagations per packet - random topology')
fig.show()

In [11]:
fig = get_propagations_per_packet_bar_chart('normal', title='Prpagations per packet - normal topology')
fig.show()

In [12]:
fig = get_propagations_per_packet_bar_chart('grid', title='Propagatios per packet - grid topology')
fig.show()

In [13]:
fig = get_propagations_per_packet_bar_chart('circle', title='Propagatios per packet - circle topology')
fig.show()

### Delays

In [14]:
fig = px.line(simulations, x='Perturbation speed', y='Average delays', color='Topology', title='Average delays', line_shape='spline')
fig.update_yaxes(title_text='Delay')
fig.update_xaxes(title_text='Speed')
fig.show()

In [15]:
fig = px.line(simulations, x='Perturbation speed', y='Maximum delays', color='Topology', title='Max delays', line_shape='spline')
fig.update_yaxes(title_text='Delay')
fig.update_xaxes(title_text='Speed')
fig.show()

In [16]:
fig = px.line(simulations, x='Perturbation speed', y='Minimum delays', color='Topology', title='Min delays', line_shape='spline')
fig.update_yaxes(title_text='Delay')
fig.update_xaxes(title_text='Speed')
fig.show()

In [17]:
fig = get_scatter_chart('random', ['Minimum delays', 'Average delays', 'Maximum delays'], title='Delays - random topology')
fig.show()

In [18]:
fig = get_scatter_chart('normal', ['Minimum delays', 'Average delays', 'Maximum delays'], title='Delays - normal topology')
fig.show()

In [19]:
fig = get_scatter_chart('grid', ['Minimum delays', 'Average delays', 'Maximum delays'], title='Delays - grid topology')
fig.show()

In [20]:
fig = get_scatter_chart('circle', ['Minimum delays', 'Average delays', 'Maximum delays'], title='Delays - circle topology')
fig.show()